In [1]:
import pandas as pd
import numpy as np
from itertools import product

In [3]:
sales = pd.read_csv('sales_train_v2.csv')

In [13]:
all_data.groupby('item_id').size()

item_id
0          50
1         252
2         101
3         101
4          50
5          50
6          50
7          50
8         101
9          51
10         50
11         50
12         46
13         50
14         50
15         50
16         50
17         50
18         51
19         45
20         51
21         50
22         47
23         47
24         47
25         51
26         52
27        739
28        595
29        321
         ... 
22140    1499
22141     192
22142     143
22143     448
22144     435
22145     492
22146     719
22147     743
22148      92
22149     594
22150     653
22151     320
22152     548
22153     338
22154     537
22155     790
22156     137
22157     182
22158      44
22159      87
22160     505
22161      45
22162     449
22163     129
22164     399
22165      94
22166    1176
22167    1221
22168     182
22169      48
Length: 21807, dtype: int64

In [33]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [25]:
all_data['count_for_grouped_ids'] = all_data.groupby('item_id')['target'].transform(np.size)
all_data

,shop_id,item_id,date_block_num,target,target_sum_for_grouped_ids,count_for_grouped_ids
139255,0,19,0,0.0,1.0,45.0
141495,0,27,0,0.0,42.0,739.0
144968,0,28,0,0.0,84.0,595.0
142661,0,29,0,0.0,12.0,321.0
138947,0,32,0,6.0,2092.0,1586.0
138948,0,33,0,3.0,836.0,1586.0
138949,0,34,0,0.0,122.0,835.0
139247,0,35,0,1.0,222.0,235.0
142672,0,40,0,0.0,94.0,1325.0
142065,0,41,0,0.0,43.0,501.0


In [22]:
all_data['target_sum_for_grouped_ids'] = all_data.groupby('item_id')['target'].transform('sum')
all_data['size_of_grouped_ids'] = all_data.groupby('item_id').transform(np.size)
all_data['LOO'] = (all_data.target_sum_for_grouped_ids - all_data.target) / (all_data.size_of_grouped_ids -1) 

ValueError: Wrong number of items passed 4, placement implies 1

In [19]:
all_data.sample(100)

,shop_id,item_id,date_block_num,target,target_sum_for_groupe_ids,target_sum_for_grouped_ids,size_of_grouped_ids,LOO
5495687,12,16362,14,0.0,85.0,85.0,903,0.094235
495105,14,14569,1,0.0,30.0,30.0,691,0.043478
6047237,48,5366,16,0.0,24.0,24.0,348,0.069164
5411854,38,19846,14,0.0,81.0,81.0,937,0.086538
1817555,43,8759,4,0.0,201.0,201.0,1230,0.163548
8535592,21,5868,24,0.0,141.0,141.0,1276,0.110588
1264017,13,17099,3,1.0,50.0,50.0,547,0.089744
10505044,44,4979,32,0.0,340.0,340.0,801,0.425000
10787250,49,4122,33,0.0,17.0,17.0,603,0.028239
5033148,18,20782,13,1.0,45.0,45.0,679,0.064897


In [29]:
all_data['group_mean'] = all_data.groupby('item_id')['target'].transform(np.mean)
all_data['nrows'] = all_data.groupby('item_id')['target'].transform(np.size)
alpha = 100
global_mean = 0.3343
all_data['smoothing'] = ((all_data.group_mean*all_data.nrows)+(global_mean*alpha))/(all_data.nrows+alpha)

encoded_feature = all_data['smoothing'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.48181987971


In [42]:
cumsum = all_data.groupby('item_id')['target'].cumsum() - all_data.target
cumcnt = all_data.groupby('item_id').cumcount()
all_data['expanding'] = cumsum /cumcnt

all_data['expanding'].fillna(0.3343, inplace=True) 

encoded_feature = all_data['expanding'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.502524521108


In [41]:
all_data.sample(100)

,shop_id,item_id,date_block_num,target,expanding
2655624,57,5294,7,0.0,0.045977
9428951,52,13124,27,0.0,0.146552
9889310,19,21528,29,0.0,0.110891
10275879,35,21622,31,0.0,0.154306
5689198,18,8714,15,0.0,0.093272
1186367,29,10888,3,1.0,0.137500
612238,8,18337,1,0.0,0.132075
3512888,54,18608,9,1.0,0.237778
9915059,12,6615,29,1.0,0.427386
5258593,56,8739,14,0.0,0.440529
